In [157]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

In [158]:
data_train = pd.read_csv('./train.csv')
data_test = pd.read_csv('./test.csv')
sample_submission = pd.read_csv('./sample_submission.csv')
print("train shape ===================================================")
print(data_train.shape)
print("train indexes Columns dtypes NaN ===================================================")
#print(data_train.info())

train shape ===================================================
(1460, 81)
train indexes Columns dtypes NaN ===================================================


In [159]:
data_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [160]:
data_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [161]:
sample_submission.head()

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977


In [162]:
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 2 columns):
Id           1459 non-null int64
SalePrice    1459 non-null float64
dtypes: float64(1), int64(1)
memory usage: 22.9 KB


In [163]:
columns = pd.read_csv('./columns.csv')

In [164]:
columns['Columns'] = columns['Columns'].astype('str')
sample_submission['SalePrice'] = sample_submission['SalePrice'].astype('float')

In [165]:
columns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 1 columns):
Columns    35 non-null object
dtypes: object(1)
memory usage: 360.0+ bytes


In [166]:
columns_flag = columns.iloc[0:6]
columns_bin = columns.iloc[7:9]
columns_category = columns.iloc[10:20]
columns_date = columns.iloc[21:24]
columns_number = columns.iloc[25:35]

In [167]:
data_train.drop(data_train.index[[1459]], inplace=True)
#data_test.drop(data_train.index[[1459]], inplace=True)
data_usefull_tr = data_train[np.concatenate((columns_flag['Columns'].values, \
                                             columns_bin['Columns'].values, \
                                             columns_category['Columns'].values, \
                                             columns_date['Columns'].values, \
                                             columns_number['Columns'].values))]
data_usefull_te = data_test[np.concatenate((columns_flag['Columns'].values, \
                                            columns_bin['Columns'].values, \
                                            columns_category['Columns'].values, \
                                            columns_date['Columns'].values, \
                                            columns_number['Columns'].values))]

Y_train = data_train[['SalePrice']]
Y_test = sample_submission[['SalePrice']]

In [168]:
data_usefull_tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 0 to 1458
Data columns (total 31 columns):
SaleType         1459 non-null object
SaleCondition    1459 non-null object
MSZoning         1459 non-null object
LandContour      1459 non-null object
Foundation       1459 non-null object
HouseStyle       1459 non-null object
Street           1459 non-null object
Alley            91 non-null object
Utilities        1459 non-null object
ExterQual        1459 non-null object
ExterCond        1459 non-null object
BsmtFinType1     1422 non-null object
HeatingQC        1459 non-null object
KitchenQual      1459 non-null object
GarageFinish     1378 non-null object
PavedDrive       1459 non-null object
PoolQC           7 non-null object
Fence            281 non-null object
YrSold           1459 non-null int64
YearRemodAdd     1459 non-null int64
YearBuilt        1459 non-null int64
OverallQual      1459 non-null int64
OverallCond      1459 non-null int64
GarageCars       1459 non-null

In [169]:
data_usefull_te.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 31 columns):
SaleType         1458 non-null object
SaleCondition    1459 non-null object
MSZoning         1455 non-null object
LandContour      1459 non-null object
Foundation       1459 non-null object
HouseStyle       1459 non-null object
Street           1459 non-null object
Alley            107 non-null object
Utilities        1457 non-null object
ExterQual        1459 non-null object
ExterCond        1459 non-null object
BsmtFinType1     1417 non-null object
HeatingQC        1459 non-null object
KitchenQual      1458 non-null object
GarageFinish     1381 non-null object
PavedDrive       1459 non-null object
PoolQC           3 non-null object
Fence            290 non-null object
YrSold           1459 non-null int64
YearRemodAdd     1459 non-null int64
YearBuilt        1459 non-null int64
OverallQual      1459 non-null int64
OverallCond      1459 non-null int64
GarageCars       1458 non-nul

In [170]:
Y_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 0 to 1458
Data columns (total 1 columns):
SalePrice    1459 non-null int64
dtypes: int64(1)
memory usage: 22.8 KB


In [171]:
data_usefull_tr['Alley'].fillna("NA", inplace=True)
data_usefull_tr['GarageFinish'].fillna("NA", inplace=True)
data_usefull_tr['PoolQC'].fillna("NA", inplace=True)
data_usefull_tr['Fence'].fillna("NA", inplace=True)
data_usefull_tr['BsmtFinType1'].fillna("NA", inplace=True)
data_usefull_tr['GarageCars'].fillna(0, inplace=True)
data_usefull_tr['GarageArea'].fillna(0, inplace=True)


/home/user/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [172]:
data_usefull_te['Alley'].fillna("NA", inplace=True)
data_usefull_te['GarageFinish'].fillna("NA", inplace=True)
data_usefull_te['PoolQC'].fillna("NA", inplace=True)
data_usefull_te['Fence'].fillna("NA", inplace=True)
data_usefull_te['BsmtFinType1'].fillna("NA", inplace=True)
data_usefull_te['Utilities'].fillna("ELO", inplace=True)
data_usefull_te['KitchenQual'].fillna("NA", inplace=True)
data_usefull_te['BsmtFinSF2'].fillna(0, inplace=True)
data_usefull_te['BsmtUnfSF'].fillna(0, inplace=True)
data_usefull_te['TotalBsmtSF'].fillna(0, inplace=True)
data_usefull_te['GarageCars'].fillna(0, inplace=True)
data_usefull_te['GarageArea'].fillna(0, inplace=True)

In [173]:
data_usefull_te.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 31 columns):
SaleType         1458 non-null object
SaleCondition    1459 non-null object
MSZoning         1455 non-null object
LandContour      1459 non-null object
Foundation       1459 non-null object
HouseStyle       1459 non-null object
Street           1459 non-null object
Alley            1459 non-null object
Utilities        1459 non-null object
ExterQual        1459 non-null object
ExterCond        1459 non-null object
BsmtFinType1     1459 non-null object
HeatingQC        1459 non-null object
KitchenQual      1459 non-null object
GarageFinish     1459 non-null object
PavedDrive       1459 non-null object
PoolQC           1459 non-null object
Fence            1459 non-null object
YrSold           1459 non-null int64
YearRemodAdd     1459 non-null int64
YearBuilt        1459 non-null int64
OverallQual      1459 non-null int64
OverallCond      1459 non-null int64
GarageCars       1459 no

In [174]:
data_flag_tr = data_usefull_tr[columns_flag['Columns'].values]
data_flag_tr = pd.get_dummies(data_flag_tr)

data_flag_te = data_usefull_te[columns_flag['Columns'].values]
data_flag_te = pd.get_dummies(data_flag_te)

data_bin_tr = data_usefull_tr[columns_bin['Columns'].values]
data_bin_tr['Street'] = np.where(data_bin_tr['Street'].str.contains("Pave"), 1, 0)
data_bin_tr['Alley'] = np.where(data_bin_tr['Alley'].str.contains("NA"), 0, 1)

data_bin_te = data_usefull_te[columns_bin['Columns'].values]
data_bin_te['Street'] = np.where(data_bin_te['Street'].str.contains("Pave"), 1, 0)
data_bin_te['Alley'] = np.where(data_bin_te['Alley'].str.contains("NA"), 0, 1)

data_category_tr = data_usefull_tr[columns_category['Columns'].values]
data_category_te = data_usefull_te[columns_category['Columns'].values]
#cleanup_education = {"education": {"Preschool": 0, "1st-4th": 1, '5th-6th': 2, '7th-8th':3, '9th':4, '10th':5,'11th':6, '12th': 7, 'HS-grad':8, 'Prof-school': 9, 'Assoc-acdm': 10, 'Assoc-voc': 11, 'Some-college': 12, 'Bachelors':13, 'Masters': 14, 'Doctorate': 15}}
#X.replace(cleanup_education, inplace = True)
cleanup_Utilities = {"Utilities": {"ELO": 0, "NoSeWa":1, "NoSewr":2, "AllPub":3}}
cleanup_ExterQual = {"ExterQual":{"Ex":4, "Gd":3, "TA":2, "Fa":1, "Po":0}}
cleanup_ExterCond = {"ExterCond":{"Ex":4, "Gd":3, "TA":2, "Fa":1, "Po":0}}
cleanup_BsmtFinType1 = {"BsmtFinType1":{"GLQ":6, "ALQ":5, "BLQ":4, "Rec":3, "LwQ":2, "Unf":1, "NA":0}}

cleanup_HeatingQC = {"HeatingQC":{"Ex":5, "Gd":4, "TA":3, "Fa":2, "Po":1, "NA": 0}}
cleanup_KitchenQual = {"KitchenQual":{"Ex":5, "Gd":4, "TA":3, "Fa":2, "Po":1, "NA": 0}}
cleanup_GarageFinish = {"GarageFinish":{"Fin":3, "RFn":2, "Unf":1, "NA":0}}
cleanup_PavedDrive = {"PavedDrive":{"Y":2, "P":1, "N":0}}
cleanup_PoolQC = {"PoolQC":{"Ex":5, "Gd":4, "TA":3, "Fa":2, "Po":1, "NA": 0, "NaN": 0}}
cleanup_Fence = {"Fence":{"GdPrv":4, "MnPrv":3, "GdWo":2, "MnWw":1, "NA":0}}

data_category_tr.replace(cleanup_Utilities, inplace = True)
data_category_tr.replace(cleanup_ExterQual, inplace = True)
data_category_tr.replace(cleanup_ExterCond, inplace = True)
data_category_tr.replace(cleanup_BsmtFinType1, inplace = True)
data_category_tr.replace(cleanup_HeatingQC, inplace = True)
data_category_tr.replace(cleanup_KitchenQual, inplace = True)
data_category_tr.replace(cleanup_GarageFinish, inplace = True)
data_category_tr.replace(cleanup_PavedDrive, inplace = True)
data_category_tr.replace(cleanup_PoolQC, inplace = True)
data_category_tr.replace(cleanup_Fence, inplace = True)

data_category_te.replace(cleanup_Utilities, inplace = True)
data_category_te.replace(cleanup_ExterQual, inplace = True)
data_category_te.replace(cleanup_ExterCond, inplace = True)
data_category_te.replace(cleanup_BsmtFinType1, inplace = True)
data_category_te.replace(cleanup_HeatingQC, inplace = True)
data_category_te.replace(cleanup_KitchenQual, inplace = True)
data_category_te.replace(cleanup_GarageFinish, inplace = True)
data_category_te.replace(cleanup_PavedDrive, inplace = True)
data_category_te.replace(cleanup_PoolQC, inplace = True)
data_category_te.replace(cleanup_Fence, inplace = True)

data_date_tr = data_usefull_tr[columns_date['Columns'].values]
data_date_te = data_usefull_te[columns_date['Columns'].values]
sc1 = StandardScaler()
data_date_tr[['YrSold', 'YearRemodAdd', 'YearBuilt']] = sc1.fit_transform(data_date_tr[['YrSold', 'YearRemodAdd', 'YearBuilt']] )
data_number_tr = data_usefull_tr[columns_number['Columns'].values]
data_number_tr = data_number_tr.astype('float')
data_number_tr[['OverallQual', 'OverallCond', 'GarageCars', 'GarageArea', 'LotArea', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'TotRmsAbvGrd', 'Fireplaces']] = sc1.fit_transform(data_number_tr[['OverallQual', 'OverallCond', 'GarageCars', 'GarageArea', 'LotArea', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'TotRmsAbvGrd', 'Fireplaces']] )

data_date_te[['YrSold', 'YearRemodAdd', 'YearBuilt']] = sc1.fit_transform(data_date_te[['YrSold', 'YearRemodAdd', 'YearBuilt']] )
data_number_te = data_usefull_te[columns_number['Columns'].values]
data_number_te = data_number_te.astype('float')
data_number_te[['OverallQual', 'OverallCond', 'GarageCars', 'GarageArea', 'LotArea', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'TotRmsAbvGrd', 'Fireplaces']] = sc1.fit_transform(data_number_te[['OverallQual', 'OverallCond', 'GarageCars', 'GarageArea', 'LotArea', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'TotRmsAbvGrd', 'Fireplaces']] )


/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

In [175]:
data_usefull_tr.drop(data_usefull_tr.index, inplace=True)
data_usefull_te.drop(data_usefull_te.index, inplace=True)
#data_flag_tr, 
data_usefull_tr = pd.concat([data_flag_tr, data_bin_tr, data_category_tr, data_date_tr, data_number_tr],axis=1)
data_usefull_te = pd.concat([data_flag_tr, data_bin_te,  data_category_te, data_date_te, data_number_te],axis=1)

/home/user/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [176]:
data_usefull_tr.head()

,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,...,OverallQual,OverallCond,GarageCars,GarageArea,LotArea,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,TotRmsAbvGrd,Fireplaces
0,0,0,0,0,0,0,0,0,1,0,...,0.650852,-0.516787,0.311027,0.350351,-0.207111,-0.287744,-0.945245,-0.458867,0.911711,-0.951848
1,0,0,0,0,0,0,0,0,1,0,...,-0.072372,2.179252,0.311027,-0.061360,-0.091895,-0.287744,-0.641887,0.466648,-0.318804,0.599824
2,0,0,0,0,0,0,0,0,1,0,...,0.650852,-0.516787,0.311027,0.631062,0.073415,-0.287744,-0.302307,-0.312973,-0.318804,0.599824
3,0,0,0,0,0,0,0,0,1,1,...,0.650852,-0.516787,1.649634,0.790132,-0.096904,-0.287744,-0.062338,-0.686827,0.296454,0.599824
4,0,0,0,0,0,0,0,0,1,0,...,1.374077,-0.516787,1.649634,1.697766,0.374980,-0.287744,-0.175531,0.199936,1.526968,0.599824


In [177]:
data_usefull_te.head()

,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,...,OverallQual,OverallCond,GarageCars,GarageArea,LotArea,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,TotRmsAbvGrd,Fireplaces
0,0,0,0,0,0,0,0,0,1,0,...,-0.751101,0.400766,-0.984704,1.185511,0.363929,0.517537,-0.649392,-0.368484,-0.918335,-0.898055
1,0,0,0,0,0,0,0,0,1,0,...,-0.054877,0.400766,-0.984704,-0.738518,0.897861,-0.297689,-0.338322,0.639542,-0.255371,-0.898055
2,0,0,0,0,0,0,0,0,1,0,...,-0.751101,-0.497418,0.302646,0.043982,0.809646,-0.297689,-0.953599,-0.264750,-0.255371,0.647066
3,0,0,0,0,0,0,0,0,1,1,...,-0.054877,0.400766,0.302646,-0.011253,0.032064,-0.297689,-0.525879,-0.269260,0.407593,0.647066
4,0,0,0,0,0,0,0,0,1,0,...,1.337571,-0.497418,0.302646,0.154452,-0.971808,-0.297689,1.059203,0.529042,-0.918335,-0.898055


In [190]:
dtr = DecisionTreeRegressor(max_depth=5, random_state=0)
knnr = KNeighborsRegressor()
lr = LinearRegression()

In [191]:
data_usefull_te.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 0 to 1458
Data columns (total 63 columns):
SaleType_COD             1459 non-null uint8
SaleType_CWD             1459 non-null uint8
SaleType_Con             1459 non-null uint8
SaleType_ConLD           1459 non-null uint8
SaleType_ConLI           1459 non-null uint8
SaleType_ConLw           1459 non-null uint8
SaleType_New             1459 non-null uint8
SaleType_Oth             1459 non-null uint8
SaleType_WD              1459 non-null uint8
SaleCondition_Abnorml    1459 non-null uint8
SaleCondition_AdjLand    1459 non-null uint8
SaleCondition_Alloca     1459 non-null uint8
SaleCondition_Family     1459 non-null uint8
SaleCondition_Normal     1459 non-null uint8
SaleCondition_Partial    1459 non-null uint8
MSZoning_C (all)         1459 non-null uint8
MSZoning_FV              1459 non-null uint8
MSZoning_RH              1459 non-null uint8
MSZoning_RL              1459 non-null uint8
MSZoning_RM              1459 non-null

In [192]:
data_usefull_tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 0 to 1458
Data columns (total 63 columns):
SaleType_COD             1459 non-null uint8
SaleType_CWD             1459 non-null uint8
SaleType_Con             1459 non-null uint8
SaleType_ConLD           1459 non-null uint8
SaleType_ConLI           1459 non-null uint8
SaleType_ConLw           1459 non-null uint8
SaleType_New             1459 non-null uint8
SaleType_Oth             1459 non-null uint8
SaleType_WD              1459 non-null uint8
SaleCondition_Abnorml    1459 non-null uint8
SaleCondition_AdjLand    1459 non-null uint8
SaleCondition_Alloca     1459 non-null uint8
SaleCondition_Family     1459 non-null uint8
SaleCondition_Normal     1459 non-null uint8
SaleCondition_Partial    1459 non-null uint8
MSZoning_C (all)         1459 non-null uint8
MSZoning_FV              1459 non-null uint8
MSZoning_RH              1459 non-null uint8
MSZoning_RL              1459 non-null uint8
MSZoning_RM              1459 non-null

In [205]:
dtr.fit(data_usefull_tr, Y_train)
knnr.fit(data_usefull_tr, Y_train)
lr.fit(data_usefull_tr, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [206]:
dtr.feature_importances_

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       5.29495555e-04, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.43285349e-02,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.36187128e-03, 2.69542919e-03, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.09591626e-02,
       7.37965636e-03, 7.83923294e-01, 0.00000000e+00, 5.15577670e-02,
      

In [207]:
dtr.score(data_usefull_te, Y_test)

-16.806944681088662

In [208]:
knnr.score(data_usefull_te, Y_test)

-13.826165969839275

In [209]:
lr.score(data_usefull_te, Y_test)

-13.865192625062024